In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_csv("LoanApprovalPrediction.csv")
data.shape

(598, 13)

In [ ]:
data.head(3)

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0.0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1.0,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0.0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 598 entries, 0 to 597
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            598 non-null    object 
 1   Gender             598 non-null    object 
 2   Married            598 non-null    object 
 3   Dependents         586 non-null    float64
 4   Education          598 non-null    object 
 5   Self_Employed      598 non-null    object 
 6   ApplicantIncome    598 non-null    int64  
 7   CoapplicantIncome  598 non-null    float64
 8   LoanAmount         577 non-null    float64
 9   Loan_Amount_Term   584 non-null    float64
 10  Credit_History     549 non-null    float64
 11  Property_Area      598 non-null    object 
 12  Loan_Status        598 non-null    object 
dtypes: float64(5), int64(1), object(7)
memory usage: 60.9+ KB


In [ ]:
data.drop(['Loan_ID'],axis=1,inplace=True)

In [ ]:
data['Loan_Status'].value_counts()

,count
Loan_Status,
Y,411
N,187


In [ ]:
#onehot encoder dummy encoding
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
one_encoder = ohe.fit(data[['Gender','Married','Education','Self_Employed','Property_Area']])
transformed = one_encoder.transform(data[['Gender','Married','Education','Self_Employed','Property_Area']])
transformed_data = pd.DataFrame(transformed.toarray(), columns = one_encoder.get_feature_names_out())
data.drop(['Gender','Married','Education','Self_Employed','Property_Area'],axis=1,inplace=True)
new_data = pd.concat([data,transformed_data],axis=1)
transformed_data.head(3)

,Gender_Female,Gender_Male,Married_No,Married_Yes,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
1,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
2,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0


In [ ]:
new_data.head(3)

,Dependents,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status,Gender_Female,Gender_Male,Married_No,Married_Yes,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
0,0.0,5849,0.0,NaN,360.0,1.0,Y,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
1,1.0,4583,1508.0,128.0,360.0,1.0,N,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
2,0.0,3000,0.0,66.0,360.0,1.0,Y,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0


In [ ]:
one_encoder.categories_

[array(['Female', 'Male'], dtype=object),
 array(['No', 'Yes'], dtype=object),
 array(['Graduate', 'Not Graduate'], dtype=object),
 array(['No', 'Yes'], dtype=object),
 array(['Rural', 'Semiurban', 'Urban'], dtype=object)]

In [ ]:
new_data.shape

(598, 18)

In [ ]:
#save encoder
import pickle
pickle.dump(one_encoder,open('loan_onehotencoder.pkl','wb'))

In [ ]:
from sklearn.preprocessing import LabelEncoder

# label_encoder object knows how
# to understand word labels.
label_encoder = LabelEncoder()
new_data['Loan_Status'] = label_encoder.fit_transform(new_data['Loan_Status'])
print(label_encoder.classes_)

['N' 'Y']


In [ ]:
new_data.describe()

,Dependents,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status,Gender_Female,Gender_Male,Married_No,Married_Yes,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
count,586.000000,598.000000,598.000000,577.000000,584.000000,549.000000,598.000000,598.000000,598.000000,598.000000,598.000000,598.000000,598.000000,598.000000,598.000000,598.000000,598.000000,598.000000
mean,0.755973,5292.252508,1631.499866,144.968804,341.917808,0.843352,0.687291,0.185619,0.814381,0.351171,0.648829,0.777592,0.222408,0.816054,0.183946,0.292642,0.376254,0.331104
std,1.007751,5807.265364,2953.315785,82.704182,65.205994,0.363800,0.463985,0.389124,0.389124,0.477736,0.477736,0.416212,0.416212,0.387765,0.387765,0.455356,0.484851,0.471004
min,0.000000,150.000000,0.000000,9.000000,12.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,2877.500000,0.000000,100.000000,360.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,3806.000000,1211.500000,127.000000,360.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
75%,1.750000,5746.000000,2324.000000,167.000000,360.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000
max,3.000000,81000.000000,41667.000000,650.000000,480.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
new_data.isna().sum()

,0
Dependents,12
ApplicantIncome,0
CoapplicantIncome,0
LoanAmount,21
Loan_Amount_Term,14
Credit_History,49
Loan_Status,0
Gender_Female,0
Gender_Male,0
Married_No,0


In [ ]:
for col in data.columns:
  new_data[col] = new_data[col].fillna(new_data[col].mean())

In [ ]:
new_data.isna().sum()

,0
Dependents,0
ApplicantIncome,0
CoapplicantIncome,0
LoanAmount,0
Loan_Amount_Term,0
Credit_History,0
Loan_Status,0
Gender_Female,0
Gender_Male,0
Married_No,0


In [ ]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
X = new_data.drop(['Loan_Status'],axis=1)
Y = new_data['Loan_Status']
X, Y = SMOTE().fit_resample(X, Y)

In [ ]:
Y.value_counts()

,count
Loan_Status,
1,411
0,411


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.4,random_state=1)
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((493, 17), (329, 17), (493,), (329,))

In [ ]:
#standardization
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
std_scaler = sc.fit(X_train)
X_train = sc.transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
#save scaler
import pickle
pickle.dump(std_scaler,open('loan_scaler.pkl','wb'))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

knn = KNeighborsClassifier(n_neighbors=3)
rfc = RandomForestClassifier(n_estimators = 7, criterion = 'entropy',random_state =7)
svc = SVC()
lc = LogisticRegression()

# making predictions on the training set
for clf in (rfc, knn, svc,lc):
    clf.fit(X_train, Y_train)
    Y_pred = clf.predict(X_train)
    print("Accuracy score of ",clf,"=",100*accuracy_score(Y_train,Y_pred))

Accuracy score of  RandomForestClassifier(criterion='entropy', n_estimators=7, random_state=7) = 98.78296146044624
Accuracy score of  KNeighborsClassifier(n_neighbors=3) = 87.4239350912779
Accuracy score of  SVC() = 84.17849898580123
Accuracy score of  LogisticRegression() = 73.63083164300203


In [ ]:
# making predictions on the testing set
for clf in (rfc, knn, svc,lc):
    clf.fit(X_train, Y_train)
    Y_pred = clf.predict(X_test)
    print("Accuracy score of ",clf,"=",100*accuracy_score(Y_test,Y_pred))

Accuracy score of  RandomForestClassifier(criterion='entropy', n_estimators=7, random_state=7) = 80.85106382978722
Accuracy score of  KNeighborsClassifier(n_neighbors=3) = 79.33130699088146
Accuracy score of  SVC() = 79.33130699088146
Accuracy score of  LogisticRegression() = 77.20364741641338


In [ ]:
final_model = RandomForestClassifier(n_estimators = 7, criterion = 'entropy',random_state =7)
final_model.fit(X_train, Y_train)

RandomForestClassifier(criterion='entropy', n_estimators=7, random_state=7)

In [ ]:
import pickle
pickle.dump(final_model,open('loanmodel.pkl','wb'))

In [ ]:
new_data.columns

Index(['Dependents', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Loan_Status', 'Gender_Female',
       'Gender_Male', 'Married_No', 'Married_Yes', 'Education_Graduate',
       'Education_Not Graduate', 'Self_Employed_No', 'Self_Employed_Yes',
       'Property_Area_Rural', 'Property_Area_Semiurban',
       'Property_Area_Urban'],
      dtype='object')

In [ ]:
#predict from saved model
import pickle
model = pickle.load(open('loanmodel.pkl','rb'))
one_encoder = pickle.load(open('loan_onehotencoder.pkl','rb'))
std_scaler = pickle.load(open('loan_scaler.pkl','rb'))
def predict_loan(Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area):
  encoded_data = one_encoder.transform([[Gender,Married,Education,Self_Employed,Property_Area]])
  encoded_data = pd.DataFrame(encoded_data.toarray(), columns = one_encoder.get_feature_names_out())
  input_data = pd.DataFrame([[Dependents,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History]],columns=['Dependents','ApplicantIncome','CoapplicantIncome','LoanAmount','Loan_Amount_Term','Credit_History'])
  encoded_data = pd.concat([input_data,encoded_data],axis=1)
  encoded_data = std_scaler.transform(encoded_data)
  print(encoded_data)
  prediction = model.predict(encoded_data)
  return prediction[0]

status = predict_loan('Male','Yes',2,'Graduate','No',4583,1508,128,360,1,'Urban')
print("yes Loan can be approved") if status == 1 else print("No Loan can be approved")

[[ 1.24981788 -0.14002518 -0.0096207  -0.2566721   0.30458909  0.56978635
  -0.50207046  0.50207046 -0.76028814  0.76028814  0.56511885 -0.56511885
   0.48987071 -0.48987071 -0.71740721 -0.81387339  1.58378092]]
yes Loan can be approved
